In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime, pickle
%matplotlib inline
import datetime

import sys
sys.path.append('../src/')
sys.path.append('../private/')

from credentials import *
from api import call_refresh_api, call_jquants_api

refreshtoken = REFRESH_TOKEN
id_token = call_refresh_api(refreshtoken)

## ニュースデータ

In [2]:
# 最初の250件を取得
idtk = id_token['idToken']
paramdict = {}
paramdict["datefrom"] = "2021-01-01"
paramdict["dateto"] = "2021-04-01"
res = call_jquants_api(paramdict, idtk, "news")
news = res['news']

while True:
    paramdict.clear()
    paramdict['scrollId'] = res['scrollId']
    res = call_jquants_api(paramdict, idtk, "news")
    if res['news'] == []:
        break
    else:
        news += res['news']    

In [3]:
df_new_article = pd.DataFrame(news)
df_new_article.to_csv('../data/new_article.csv')
len(df_new_article)

37104

In [4]:
len(set(df_new_article['article_id']))

37104

重複なく取得できている。

## テキストデータをBertを利用して分散表現に変換

In [5]:
df_new_article = pd.read_csv('../data/new_article.csv')
df_new_article.isnull().any()

Unnamed: 0          False
article_id          False
publish_datetime    False
media_code          False
media_name          False
men_name             True
headline            False
keywords             True
classifications      True
stock_code           True
dtype: bool

In [6]:
from SentimentGenerator import SentimentGenerator

sg = SentimentGenerator()
sg.initialize()

[+] Set Device: CPU
[+] Built feature extractor
[+] Built bert tokenizer


ModuleNotFoundError: You need to install fugashi to use MecabTokenizer.See https://pypi.org/project/fugashi/ for installation.  
⇒"pip install fugashi"  

ModuleNotFoundError: The ipadic dictionary is not installed. See https://github.com/polm/ipadic-py for installation.  
⇒"pip install ipadic"

In [7]:
# SentimentGenerator使用する全体流れを記述
articles = sg.load_articles(path='../data/new_article.csv')
articles = sg.normalize_articles(articles)
articles = sg.handle_punctuations_in_articles(articles)
articles = sg.drop_remove_list_words(articles)

# 以下のコードでコーパス全体の特徴量を抽出できる。しかし、抽出には長い時間が要求されるため、注意。
##headline_features = SentimentGenerator.generate_features_by_texts(texts=articles['headline'])
#keywords_features = SentimentGenerator.generate_features_by_texts(texts=articles['keywords'])

In [8]:
headline_features = sg.generate_features_by_texts(texts=articles['headline'])

KeyboardInterrupt: 

⇒GoogleColaboratoryで実行する